# Better Curbs Project: Emissions Metric Calculator

### This notebook can be used to generate the Emissions Index (EI) metric for a single simulation run.

#### Developed by Tom Maxner, University of Washington
#### Last updated: 12/12/2022

### The sequence of operations in this notebook are as follows:
#### 1. Load script for .fzp (vehicle trajectory) file parsing tool, and modules for analyzing data;
#### 2. Load and parse select .fzp file, import emissions factor dataframe; 
#### 3. Calculate emissions from ignition after parking;
#### 4. Calculate emissions from idling while parking;
#### 5. Calculate total parking emissions;
#### 6. Calculate driving emissions based on driving status;
#### 7. Export processed data to .csv for fruther analysis.

### NOTE: User must update directory and file names in Steps 2 and 7. 

## Step 1. Load script for .fzp file parsing tool, and modules for analyzing data.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#passive library that generates nicer looking plots
import seaborn
seaborn.set()

In [ ]:
def pandas_parse_fzp_vissim_vehicle_records(filename):
    #this function parses a vehicle records fzp file and 
    #reads the data into a pandas dataframe
    
    #FZP file notes
    #lines beginning with * are comments
    #escape character \ is not an escape character (VISSIM sucks)
    #line beginning with $ is header
    #data is ; delimited
    #dataframe will come back with all strings as data

    data = []
    
    with open(filename, 'r') as d:
        for line in d:
            if line[0] == "*":
                pass
            elif line[0:8] == "$VEHICLE": #column header doesn't precisely correspond to columns
                header = line.replace("\\","_").strip().split(";") #VISSIM uses stupid escape characters in stupid places
                header[0] = header[0].split(":")[1] #getting rid of garbage at start of column header
            elif line[0] == "$":
                pass
            else:       
                data.append(line.strip().split(";"))
                
    out_frame = pd.DataFrame(data)
    out_frame = out_frame.drop(labels=0, axis=0) #ends up reading first line anyway because of garbage unicode
    out_frame.columns = header
    
    return(out_frame)

## Step 2. Load and parse select .fzp file, import emissions factor dataframe

In [ ]:
#Import FZP / vehicle records file

datapath = "D:\\VISSIM_Sims_05_13_2_Lane\\Scen1" #path to the location of the FZP file you want to read / Directory
filename = "Seattle_Atomic_Network_Scen1_experiment_1_001.fzp" #FZP file you want to read

SCEN = filename[27:28]
EX = filename[40:-8] ## might have to change for single digits

S01_02 = pandas_parse_fzp_vissim_vehicle_records(os.path.join(datapath, filename))

In [ ]:
#Import emissions data
Emissions_df = pd.read_csv(r'C://Users//type//your//directory//here//Emissions_Factors.csv')
Emissions_df

In [ ]:
# Change variable columns to STRING

S01_02["ACCELERATION"] = S01_02["ACCELERATION"].astype(str).astype(float)
S01_02["VEHTYPE"] = S01_02["VEHTYPE"].astype(str).astype(float)
S01_02["SPEED"] = S01_02["SPEED"].astype(str).astype(float)
S01_02["DWELLTM"] = S01_02["DWELLTM"].astype(str).astype(float)

## Step 3. Calculate emissions from ignition after parking
##### To do this: get Maximum dwell time for each unique vehicle...

In [ ]:
# We will assume that any vehicle that parks for greater than 5 minutes (300 sec) turns their vehicle off.

In [ ]:
#Filter for dwell times >= 300 sec
Ignition_Filter = (S01_02['DWELLTM'] >= 300)
Ignition_df = S01_02[Ignition_Filter]

In [ ]:
#Add Column to Ignition_df for emissions rate category. In this case, all cells will be labelled "ignition"
Ignition_df['CATEGORY'] = 'Ignition'
#Ignition_df

In [ ]:
# Merge emissions rates onto Ignition_df on vehtype and emission rate for ignition status
IGN_EM_df = Ignition_df.merge(Emissions_df,left_on=['VEHTYPE','CATEGORY'], right_on=['VEHTYPE','CATEGORY'], how='left')
#IGN_EM_df

In [ ]:
# Calculate total emissions from ignition
IGN_EM = IGN_EM_df['EM_RATE'].sum()

print(IGN_EM, 'grams')

## Step 4. Calculate emissions from idling while parking

In [ ]:
#Filter for dwell times <= 300 sec
ParkIdle_Filter = (S01_02['DWELLTM'] <= 300)
Park_S01_02 = S01_02[ParkIdle_Filter]

In [ ]:
# The first vehicle record is the maximum dwell time
Park_S01_02 = Park_S01_02.drop_duplicates(subset = ["NO"])

In [ ]:
#Add Column to Ignition_df for emissions rate category. In this case, all cells will be labelled "Parked_Idle"
Park_S01_02['CATEGORY'] = 'Parked_Idle'
#Park_S01_02

In [ ]:
# Merge emissions rates onto ParkIdle_df on vehtype and emission rate for parked idling status
PI_EM_df = Park_S01_02.merge(Emissions_df,left_on=['VEHTYPE','CATEGORY'], right_on=['VEHTYPE','CATEGORY'], how='left')

In [ ]:
PI_EM_df['PARKIDLEEM'] = PI_EM_df['DWELLTM'] * PI_EM_df['EM_RATE']
#PI_EM_df

In [ ]:
# Calculate total emissions from idling: This should be EM_RATE x DWELLTM
PI_EM = PI_EM_df['PARKIDLEEM'].sum()

print(PI_EM, 'grams')

## Step 5. Calculate total parking emissions

In [ ]:
#Total Parking Emissions
TOT_PARK_EM = PI_EM + IGN_EM
print(TOT_PARK_EM)

In [ ]:
# Separate by Blockface
#Filter vehicle records to show only those records traveling through study area blockfaces

BF1_Filter_Idle_S01_02 = (PI_EM_df['LANE_LINK_NO'] == '1')
BF1_Filter_Ignition_S01_02 = (IGN_EM_df['LANE_LINK_NO'] == '1')
BF2_Filter_Idle_S01_02 = (PI_EM_df['LANE_LINK_NO'] == '2')
BF2_Filter_Ignition_S01_02 = (IGN_EM_df['LANE_LINK_NO'] == '2')

In [ ]:
#Create dataframes for each blockface's vehicle records

BF1_Park = PI_EM_df[BF1_Filter_Idle_S01_02]
BF1_Ign = IGN_EM_df[BF1_Filter_Ignition_S01_02]
BF2_Park = PI_EM_df[BF2_Filter_Idle_S01_02]
BF2_Ign = IGN_EM_df[BF2_Filter_Ignition_S01_02]

In [ ]:
# Define variables to update CSV

Park_Em_100_B1 = (BF1_Park[(BF1_Park['VEHTYPE'] == 100)]['PARKIDLEEM'].sum()) + (BF1_Ign[(BF1_Ign['VEHTYPE'] == 100)]['EM_RATE'].sum())
Park_Em_200_B1 = (BF1_Park[(BF1_Park['VEHTYPE'] == 200)]['PARKIDLEEM'].sum()) + (BF1_Ign[(BF1_Ign['VEHTYPE'] == 200)]['EM_RATE'].sum())
Park_Em_640_B1 = (BF1_Park[(BF1_Park['VEHTYPE'] == 640)]['PARKIDLEEM'].sum()) + (BF1_Ign[(BF1_Ign['VEHTYPE'] == 640)]['EM_RATE'].sum())
Park_Em_650_B1 = (BF1_Park[(BF1_Park['VEHTYPE'] == 650)]['PARKIDLEEM'].sum()) + (BF1_Ign[(BF1_Ign['VEHTYPE'] == 650)]['EM_RATE'].sum())
Park_Em_Total_B1 = (BF1_Park['PARKIDLEEM'].sum()) + (BF1_Ign['EM_RATE'].sum())

Park_Em_100_B2 = (BF2_Park[(BF2_Park['VEHTYPE'] == 100)]['PARKIDLEEM'].sum()) + (BF2_Ign[(BF2_Ign['VEHTYPE'] == 100)]['EM_RATE'].sum())
Park_Em_200_B2 = (BF2_Park[(BF2_Park['VEHTYPE'] == 200)]['PARKIDLEEM'].sum()) + (BF2_Ign[(BF2_Ign['VEHTYPE'] == 200)]['EM_RATE'].sum())
Park_Em_640_B2 = (BF2_Park[(BF2_Park['VEHTYPE'] == 640)]['PARKIDLEEM'].sum()) + (BF2_Ign[(BF2_Ign['VEHTYPE'] == 640)]['EM_RATE'].sum())
Park_Em_650_B2 = (BF2_Park[(BF2_Park['VEHTYPE'] == 650)]['PARKIDLEEM'].sum()) + (BF2_Ign[(BF2_Ign['VEHTYPE'] == 650)]['EM_RATE'].sum())
Park_Em_Total_B2 = (BF2_Park['PARKIDLEEM'].sum()) + (BF2_Ign['EM_RATE'].sum())

## Step 6. Calculate driving emissions based on driving status

In [ ]:
Drive_Filter = (S01_02['PARKSTATE'] != 'Parked')
Drive_df = S01_02[Drive_Filter]

In [ ]:
# Add Emission CATEGORY column
conditionsEmStat = [
    (Drive_df['ACCELERATION'] < 0), #braking 
    (Drive_df['ACCELERATION'] == 0), #idling or off
    (Drive_df['ACCELERATION'] > 0) & (Drive_df['ACCELERATION'] < 0.75) & (Drive_df['SPEED'] > 26), # 30MPH STEADY STATE DRIVING 
    (Drive_df['ACCELERATION'] > 0) & (Drive_df['ACCELERATION'] < 0.75) & (Drive_df['SPEED'] >= 20) & (Drive_df['SPEED'] <= 26), # 20MPH STEADY STATE DRIVING
    (Drive_df['ACCELERATION'] >= 6) & (Drive_df['SPEED'] >= 0) & (Drive_df['SPEED'] < 10), #FAST ACCELERATION 0-10 mph
    (Drive_df['ACCELERATION'] >= 6) & (Drive_df['SPEED'] >= 10) & (Drive_df['SPEED'] < 20), #FAST ACCELERATION 10-20 mph
    (Drive_df['ACCELERATION'] >= 6) & (Drive_df['SPEED'] >= 20) & (Drive_df['SPEED'] < 30), #FAST ACCELERATION 20-30 mph
    (Drive_df['ACCELERATION'] >= 6) & (Drive_df['SPEED'] >= 30), #FAST ACCELERATION 30+ mph
    (Drive_df['ACCELERATION'] > 1.75) & (Drive_df['ACCELERATION'] < 6) & (Drive_df['SPEED'] >= 0) & (Drive_df['SPEED'] < 10), #MODERATE ACCELERATION 0-10 mph
    (Drive_df['ACCELERATION'] > 1.75) & (Drive_df['ACCELERATION'] < 6) & (Drive_df['SPEED'] >= 10) & (Drive_df['SPEED'] < 20), #MODERATE ACCELERATION 10-20 mph
    (Drive_df['ACCELERATION'] > 1.75) & (Drive_df['ACCELERATION'] < 6) & (Drive_df['SPEED'] >= 20) & (Drive_df['SPEED'] < 30), #MODERATE ACCELERATION 20-30 mph
    (Drive_df['ACCELERATION'] > 1.75) & (Drive_df['ACCELERATION'] < 6) & (Drive_df['SPEED'] >= 30), #MODERATE ACCELERATION 30+ mph
    (Drive_df['ACCELERATION'] >= 0.75) & (Drive_df['ACCELERATION'] <= 1.75) & (Drive_df['SPEED'] >= 0) & (Drive_df['SPEED'] < 10), #SLOW ACCELERATION 0-10 mph
    (Drive_df['ACCELERATION'] >= 0.75) & (Drive_df['ACCELERATION'] <= 1.75) & (Drive_df['SPEED'] >= 10) & (Drive_df['SPEED'] < 20), #SLOW ACCELERATION 10-20 mph
    (Drive_df['ACCELERATION'] >= 0.75) & (Drive_df['ACCELERATION'] <= 1.75) & (Drive_df['SPEED'] >= 20) & (Drive_df['SPEED'] < 30), #SLOW ACCELERATION 20-30 mph
    (Drive_df['ACCELERATION'] >= 0.75) & (Drive_df['ACCELERATION'] <= 1.75) & (Drive_df['SPEED'] >= 30), #SLOW ACCELERATION 30+ mph
    (Drive_df['ACCELERATION'] >= 0.01) & (Drive_df['ACCELERATION'] < 0.75) & (Drive_df['SPEED'] >= 0) & (Drive_df['SPEED'] < 10), #SLOW ACCELERATION 0-10 mph
    (Drive_df['ACCELERATION'] >= 0.01) & (Drive_df['ACCELERATION'] < 0.75) & (Drive_df['SPEED'] >= 10) & (Drive_df['SPEED'] < 20) #SLOW ACCELERATION 10-20 mph
]
choicesEmStat = ['Braking', 'Stopped_Idle', 'Steady_state_30mph', 'Steady_state_20mph', 'Fast_Acceleration_0_10', 
                 'Fast_Acceleration_10_20', 'Fast_Acceleration_20_30', 'Fast_Acceleration_30_plus',
                 'Mod_Acceleration_0_10', 'Mod_Acceleration_10_20', 'Mod_Acceleration_20_30', 'Mod_Acceleration_30_plus', 
                 'Slow_Acceleration_0_10', 'Slow_Acceleration_10_20', 'Slow_Acceleration_20_30', 'Slow_Acceleration_30_plus',
                 'Slow_Acceleration_0_10', 'Slow_Acceleration_10_20',]
Drive_df['CATEGORY'] = np.select(conditionsEmStat, choicesEmStat, default=0)

#Drive_df

In [ ]:
# Merge emissions rates onto Drive_df on vehtype and emission rate for parked idling status
DR_EM_df = Drive_df.merge(Emissions_df,left_on=['VEHTYPE','CATEGORY'], right_on=['VEHTYPE','CATEGORY'], how='left')
#DR_EM_df

In [ ]:
DR_EM_df['DRIVEEEM'] = DR_EM_df['EM_RATE'] * 0.1 # trajectories were collected in 0.1 sec increments whereas emission rates are g/sec.
#DR_EM_df

In [ ]:
#Filter vehicle records to show only those records traveling through study area blockfaces
BF1_Filter_S01_02 = (DR_EM_df['LANE_LINK_NO'] == '1')
BF2_Filter_S01_02 = (DR_EM_df['LANE_LINK_NO'] == '2')

In [ ]:
#Create dataframes for each blockface's vehicle records
BF1 = DR_EM_df[BF1_Filter_S01_02]
BF2 = DR_EM_df[BF2_Filter_S01_02]

In [ ]:
# Define variables to update CSV

Em_100_B1 = BF1[(BF1['VEHTYPE'] == 100)]['DRIVEEEM'].sum()
Em_200_B1 = BF1[(BF1['VEHTYPE'] == 200)]['DRIVEEEM'].sum()
Em_300_B1 = BF1[(BF1['VEHTYPE'] == 300)]['DRIVEEEM'].sum()
Em_640_B1 = BF1[(BF1['VEHTYPE'] == 640)]['DRIVEEEM'].sum()
Em_650_B1 = BF1[(BF1['VEHTYPE'] == 650)]['DRIVEEEM'].sum()
Em_Total_B1 = BF1['DRIVEEEM'].sum()

Em_100_B2 = BF2[(BF2['VEHTYPE'] == 100)]['DRIVEEEM'].sum()
Em_200_B2 = BF2[(BF2['VEHTYPE'] == 200)]['DRIVEEEM'].sum()
Em_300_B2 = BF2[(BF2['VEHTYPE'] == 300)]['DRIVEEEM'].sum()
Em_640_B2 = BF2[(BF2['VEHTYPE'] == 640)]['DRIVEEEM'].sum()
Em_650_B2 = BF2[(BF2['VEHTYPE'] == 650)]['DRIVEEEM'].sum()
Em_Total_B2 = BF2['DRIVEEEM'].sum()

TOTAL_B1 = Em_Total_B1 + Park_Em_Total_B1
TOTAL_B2 = Em_Total_B2 + Park_Em_Total_B2

## Step 7. Export processed data to .csv for fruther analysis

In [ ]:
# Pre-requisite - Import the DictWriter class from csv  module
from csv import DictWriter
  
# The list of column names as mentioned in the CSV file
headersCSV = ['EXP', 'FILE', 'SCEN','TOTAL_B1','TOTAL_B2', 
              'Park_Em_100_B1','Park_Em_200_B1','Park_Em_640_B1','Park_Em_650_B1','Park_Em_Total_B1',
              'Park_Em_100_B2','Park_Em_200_B2','Park_Em_640_B2','Park_Em_650_B2','Park_Em_Total_B2',
              'Em_100_B1','Em_200_B1','Em_300_B1','Em_640_B1','Em_650_B1','Em_Total_B1',
              'Em_100_B2','Em_200_B2','Em_300_B2','Em_640_B2','Em_650_B2','Em_Total_B2'              
             ]      
# The data assigned to the dictionary
# update every time: 'EXP', 'FILE', 'SCEN'
dict={'EXP':EX, 'FILE':filename, 'SCEN':SCEN,'TOTAL_B1':TOTAL_B1,'TOTAL_B2':TOTAL_B2,
      'Park_Em_100_B1':Park_Em_100_B1,'Park_Em_200_B1':Park_Em_200_B1,'Park_Em_640_B1':Park_Em_640_B1,'Park_Em_650_B1':Park_Em_650_B1,'Park_Em_Total_B1':Park_Em_Total_B1,
      'Park_Em_100_B2':Park_Em_100_B2,'Park_Em_200_B2':Park_Em_200_B2,'Park_Em_640_B2':Park_Em_640_B2,'Park_Em_650_B2':Park_Em_650_B2,'Park_Em_Total_B2':Park_Em_Total_B2,
      'Em_100_B1':Em_100_B1,'Em_200_B1':Em_200_B1,'Em_300_B1':Em_300_B1,'Em_640_B1':Em_640_B1,'Em_650_B1':Em_650_B1,'Em_Total_B1':Em_Total_B1,
      'Em_100_B2':Em_100_B2,'Em_200_B2':Em_200_B2,'Em_300_B2':Em_300_B2,'Em_640_B2':Em_640_B2,'Em_650_B2':Em_650_B2,'Em_Total_B2':Em_Total_B2
     }
  
# Pre-requisite - The CSV file should be manually closed before running this code.

# First, open the old CSV file in append mode, hence mentioned as 'a'
# Then, for the CSV file, create a file object
with open('C://Users//type//your//directory//here//TypeFilenameHere.csv', 'a', newline='') as f_object:
    # Pass the CSV  file object to the Dictwriter() function
    # Result - a DictWriter object
    dictwriter_object = DictWriter(f_object, fieldnames=headersCSV)
    # Pass the data in the dictionary as an argument into the writerow() function
    dictwriter_object.writerow(dict)
    # Close the file object
    f_object.close()

In [ ]:
print('done')